In [1]:
! pip install -U opencv-contrib-python

In [2]:
from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/drive', force_remount=True)

import os
from google.colab.patches import cv2_imshow
from tqdm.notebook import tqdm

Mounted at /content/drive


In [3]:
import sys
curr_dir = r'/content/drive/My Drive/course_ocr/task3'
if curr_dir not in sys.path:
  print(curr_dir, 'added to sys.path')
  sys.path.append(curr_dir)
else:
  print(curr_dir, 'in sys.path')

/content/drive/My Drive/course_ocr/task3 added to sys.path


In [4]:
# your path to data
train_path = r'/content/drive/My Drive/CourseOCRTask3/Train'
test_path = r'/content/drive/My Drive/CourseOCRTask3/Test'

In [5]:
markup_path = r'/content/drive/My Drive/course_ocr/task3/markup.csv'
import pandas as pd
df = pd.read_csv(markup_path, encoding='utf16', header=None, names=['File','Code', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'Extra'])

In [29]:
df_out = pd.DataFrame(index=df.index, columns=df.columns)

import cv2

In [30]:
import imutils 
import numpy as np
def get_bb(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    h, w = gray.shape

    gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 31, 15)

    gradX = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
    gradY = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=-1)

    gradient = cv2.subtract(gradX, gradY)
    gradient = cv2.convertScaleAbs(gradient)

    blurred = cv2.blur(gradient, (5, 5))
    (_, thresh) = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(w / 50), int(h / 50)))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    closed = cv2.erode(closed, None, iterations=10)
    closed = cv2.dilate(closed, None, iterations=10)

    cnts = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    cnts = imutils.grab_contours(cnts)
    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)

        left = list(map(int, c[c[:, :, 0].argmin()][0]))
        right = list(map(int, c[c[:, :, 0].argmax()][0]))
        top = list(map(int, c[c[:, :, 1].argmin()][0]))
        bot = list(map(int, c[c[:, :, 1].argmax()][0]))
        return left, top,  right, bot
    else:
        return np.zeros((4, 2), dtype=int)

In [31]:
get_bb(img)

([456, 400], [1569, 288], [1656, 723], [579, 895])

In [32]:
correct = 0 
for line in tqdm(range(df.shape[0])):
    image_path = test_path + '/Images/' + df.iloc[line]['File']
    image=cv2.imread(image_path)
    points = get_bb(image)
    df_out.iloc[line]['File'] = df.iloc[line]['File']
    df_out.iloc[line]['Code'] = 123
    df_out.iloc[line]['x1']   = points[1][0]
    df_out.iloc[line]['y1']   = points[1][1]
    df_out.iloc[line]['x2']   = points[2][0]
    df_out.iloc[line]['y2']   = points[2][1]
    df_out.iloc[line]['x3']   = points[3][0]
    df_out.iloc[line]['y3']   = points[3][1]
    df_out.iloc[line]['x4']   = points[0][0]
    df_out.iloc[line]['y4']   = points[0][1]
    # print(points )
    # print(df.iloc[line]['x1':'y4'])

  0%|          | 0/100 [00:00<?, ?it/s]

([0, 867], [555, 397], [1783, 1216], [345, 1659])
x1     569
y1     485
x2    1644
y2     452
x3    1707
y3    1335
x4     583
y4    1356
Name: 0, dtype: object
([506, 1056], [709, 374], [1890, 780], [1269, 1345])
x1     686
y1     446
x2    1833
y2     528
x3    1747
y3    1338
x4     611
y4    1117
Name: 1, dtype: object
([0, 85], [139, 0], [1605, 764], [689, 984])
x1     536
y1     483
x2    1552
y2     313
x3    1608
y3     767
x4     620
y4     939
Name: 2, dtype: object
([469, 672], [1507, 234], [1563, 630], [602, 698])
x1     522
y1     270
x2    1541
y2     229
x3    1566
y3     636
x4     537
y4     688
Name: 3, dtype: object
([441, 794], [556, 244], [1762, 853], [1727, 853])
x1     553
y1     248
x2    1648
y2     265
x3    1637
y3     745
x4     549
y4     736
Name: 4, dtype: object
([420, 771], [1357, 269], [1388, 755], [427, 824])
x1     462
y1     302
x2    1381
y2     270
x3    1387
y3     761
x4     478
y4     810
Name: 5, dtype: object
([220, 309], [884, 270], [976, 14

In [33]:
df_out.to_csv('/content/drive/My Drive/course_ocr/task3/answer.csv', header=None, encoding='utf16', index=None)

In [34]:
! cd drive/MyDrive/course_ocr/task3/course_ocr_t3; python -m evaluate

Checking answer (/content/drive/MyDrive/course_ocr/task3/answer.csv) against markup(/content/drive/MyDrive/course_ocr/task3/markup.csv)
recognition_accuracy= 0.0
detection_iou= 0.5580047529504444
score= 0.19530166353265552


In [ ]:
# это по старой метрике